In [1]:
# %pip install neo4j langchain langchain_openai langchain-community python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
import dotenv
dotenv.load_dotenv('.env', override=True)

True

In [3]:
import os
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=os.environ['NEO4J_URI'], 
    username=os.environ['NEO4J_USERNAME'],
    password=os.environ['NEO4J_PASSWORD'],
)

In [4]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

q_load_articles = """
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/dcarpintero/generative-ai-101/main/dataset/synthetic_articles.csv' 
AS row 
FIELDTERMINATOR ';'
MERGE (a:Article {title:row.Title})
SET a.abstract = row.Abstract,
    a.publication_date = date(row.Publication_Date)
FOREACH (researcher in split(row.Authors, ',') | 
    MERGE (p:Researcher {name:trim(researcher)})
    MERGE (p)-[:PUBLISHED]->(a))
FOREACH (topic in [row.Topic] | 
    MERGE (t:Topic {name:trim(topic)})
    MERGE (a)-[:IN_TOPIC]->(t))
"""

graph.query(q_load_articles)

[]

In [5]:
graph.refresh_schema()

In [6]:
print(graph.get_schema)

Node properties:
Article {title: STRING, abstract: STRING, publication_date: DATE}
Researcher {name: STRING}
Topic {name: STRING}
Relationship properties:

The relationships:
(:Article)-[:IN_TOPIC]->(:Topic)
(:Researcher)-[:PUBLISHED]->(:Article)


In [7]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=os.environ['NEO4J_URI'],
    username=os.environ['NEO4J_USERNAME'],
    password=os.environ['NEO4J_PASSWORD'],
    index_name='articles',
    node_label="Article",
    text_node_properties=['topic', 'title', 'abstract'],
    embedding_node_property='embedding',
)

In [8]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=os.environ['NEO4J_URI'],
    username=os.environ['NEO4J_USERNAME'],
    password=os.environ['NEO4J_PASSWORD'],
    index_name='articles',
    node_label="Article",
    text_node_properties=['topic', 'title', 'abstract'],
    embedding_node_property='embedding',
)

In [12]:
%pip install -U langchain-classic langchain-openai langchain-community

from langchain_classic.chains import RetrievalQA
from langchain_openai import ChatOpenAI

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.6 MB/s eta 0:00:00-:--:--
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.3
    Uninstalling langchain-core-1.1.3:
      Successfully uninstalled langchain-core-1.1.3
  Attempting uninstall: langchain-text-splitters 0/4 [langchain-core]
    Found existing installation: langchain-text-splitters 1.0.0langchain-core]
    Uninstalling langchain-text-splitters-1.0.0: 0/4 [langchain-core]
      Successfully uninstalled langchain-text-splitters-1.0.0 [langchain-core]
  Attempting uninstall: langchain-openai━━━━ 0/4 [langchain-core]
    Found existing installation: langchain-openai 1.1.1/4 [langchain-core]
    Uninstalling langchain-openai-1.1.1:━━━━ 0/4 [langchain-core]
      Successfully uninstalled langchain-openai-1.1.1m0/4 [langchain-core]
  Attempting uninstall: langchain-classic━━━ 0/4 [langchain-core]
    Found existing installation: langchain-classic 1.0.04 [langchain-core]
    Uninstal

In [13]:
r = vector_qa.invoke(
    {"query": "which articles discuss how AI might affect our daily life? include the article titles and abstracts."}
)
print(r['result'])

The articles that discuss how AI might affect our daily life are:

1. The Impact of AI on Employment: A Comprehensive Study
   Abstract: This study analyzes the potential effects of AI on various job sectors and suggests policy recommendations to mitigate negative impacts.

2. The Societal Implications of Advanced AI: A Multidisciplinary Analysis
   Abstract: Our study brings together experts from various fields to analyze the potential long-term impacts of advanced AI on society, economy, and culture.


In [16]:
from langchain_classic.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name='gpt-4o'),
    qa_llm = ChatOpenAI(temperature=0, model_name='gpt-4o'), 
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True)

In [17]:
cypher_chain.invoke(
    {"query": "How many articles has published Emily Chen?"}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (r:Researcher {name: "Emily Chen"})-[:PUBLISHED]->(a:Article)
RETURN COUNT(a) AS numberOfArticles

Full Context:
[{'numberOfArticles': 7}]

> Finished chain.


{'query': 'How many articles has published Emily Chen?',
 'result': 'Emily Chen has published 7 articles.'}